In [1]:
import pandas as pd

In [2]:
data_path = "../../../data"

prc_vazao = f"{data_path}/vazao/processed"
prc_merge = f"{data_path}/merge/processed"
datasets = f"{data_path}/datasets"
vazao_diaria_file_path = f'{prc_vazao}/vazao_diaria.csv'

bacia_incremental_file_path = f"{prc_merge}/MERGE_PREC_SUM_ITAIPU_INCREMENTAL_DAILY.csv"

### Vazão Itaipu

In [3]:
vazao_diaria = pd.read_csv(vazao_diaria_file_path, index_col='DATA') # aqui cada usina tem um range diferente

usina = 'ITAIPU (266)'
itaipu = vazao_diaria[[usina]]
itaipu.index = pd.to_datetime(itaipu.index)
itaipu['weekday'] = itaipu.index.strftime('%A')
itaipu.rename(columns={'ITAIPU (266)' : 'vazao_itaipu'}, inplace=True)
itaipu = itaipu[itaipu.index >= '1931-01-03']
itaipu

<ipython-input-3-5b5a5da2e439>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  itaipu['weekday'] = itaipu.index.strftime('%A')
<ipython-input-3-5b5a5da2e439>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  itaipu.rename(columns={'ITAIPU (266)' : 'vazao_itaipu'}, inplace=True)


,vazao_itaipu,weekday
DATA,,
1931-01-03,22091.0,Saturday
1931-01-04,21369.0,Sunday
1931-01-05,20808.0,Monday
1931-01-06,20183.0,Tuesday
1931-01-07,19093.0,Wednesday
...,...,...
2021-12-27,9664.0,Monday
2021-12-28,9825.0,Tuesday
2021-12-29,9846.0,Wednesday


In [4]:
all_rows = itaipu.shape[0]
new_itaipu = pd.DataFrame(columns=['vazao_itaipu', 'time'])
semana_operativa = 0

while semana_operativa < all_rows:

    vazao_sum = itaipu[semana_operativa:semana_operativa+7]['vazao_itaipu'].sum() # soma acumulada
    timedate_day = itaipu[semana_operativa:semana_operativa+7].index[0] # dia de sábado

    row_to_append = pd.DataFrame([{'vazao_itaipu': vazao_sum, 'time': timedate_day}])
    new_itaipu = pd.concat([new_itaipu, row_to_append])

    semana_operativa+=7

new_itaipu = new_itaipu.set_index(['time'])
new_itaipu.index = pd.to_datetime(new_itaipu.index)
new_itaipu

,vazao_itaipu
time,
1931-01-03,141236.0
1931-01-10,111756.0
1931-01-17,107046.0
1931-01-24,89510.0
1931-01-31,109602.0
...,...
2021-11-27,57676.0
2021-12-04,52667.0
2021-12-11,45549.0


In [5]:
new_itaipu = new_itaipu[(new_itaipu.index >= "2001-01-06") & (new_itaipu.index <= "2021-01-02")]
new_itaipu

,vazao_itaipu
time,
2001-01-06,96628.00
2001-01-13,113552.00
2001-01-20,84168.00
2001-01-27,81859.00
2001-02-03,98723.00
...,...
2020-12-05,37020.23
2020-12-12,46404.99
2020-12-19,63216.66


### Bacia Incremental Itaipu

In [6]:
bacia_incremental = ( 
    pd.read_csv(bacia_incremental_file_path, header=None)
    .rename(columns={0 : 'time', 1: 'bacia_prec_sum'}) # bacia_inc_prec_sum
) 

bacia_incremental.time = pd.to_datetime(bacia_incremental.time)
bacia_incremental.set_index(['time'], inplace=True)

# Uma semana operativa vai de sábado até sexta (ambos inclusos), a coluna 'time' inicia-se pelo sábado
bacia_incremental['weekday'] = bacia_incremental.index.strftime('%A')

# P/ começar no sábado e acabar no sábado
bacia_incremental = bacia_incremental[(bacia_incremental.index >= '2000-06-03') & (bacia_incremental.index <= '2023-12-29')]
bacia_incremental

,bacia_prec_sum,weekday
time,,
2000-06-03,4369.125,Saturday
2000-06-04,9345.750,Sunday
2000-06-05,11784.500,Monday
2000-06-06,331.000,Tuesday
2000-06-07,11.250,Wednesday
...,...,...
2023-12-25,2820.625,Monday
2023-12-26,13341.000,Tuesday
2023-12-27,1.250,Wednesday


In [7]:
all_rows = bacia_incremental.shape[0]
new_bacia_incremental = pd.DataFrame(columns=['bacia_prec_sum', 'time'])
semana_operativa = 0

while semana_operativa < all_rows:

    bacia_prec_sum = bacia_incremental[semana_operativa:semana_operativa+7]['bacia_prec_sum'].sum() # soma acumulada
    timedate_day = bacia_incremental[semana_operativa:semana_operativa+7].index[0] # dia de sábado

    row_to_append = pd.DataFrame([{'bacia_prec_sum': bacia_prec_sum, 'time': timedate_day}])
    new_bacia_incremental = pd.concat([new_bacia_incremental, row_to_append])

    semana_operativa+=7

new_bacia_incremental = new_bacia_incremental.set_index(['time'])
new_bacia_incremental.index = pd.to_datetime(new_bacia_incremental.index)
new_bacia_incremental

,bacia_prec_sum
time,
2000-06-03,26850.1250
2000-06-10,6488.0000
2000-06-17,75021.0000
2000-06-24,50219.3750
2000-07-01,14068.2500
...,...
2023-11-25,73645.3750
2023-12-02,63812.9375
2023-12-09,35518.6250


In [8]:
new_bacia_incremental = new_bacia_incremental[(new_bacia_incremental.index >= "2001-01-06") & (new_bacia_incremental.index <= "2021-01-02")]
new_bacia_incremental

,bacia_prec_sum
time,
2001-01-06,94990.3125
2001-01-13,56717.3750
2001-01-20,50552.3750
2001-01-27,55272.6250
2001-02-03,84943.3125
...,...
2020-12-05,44634.2500
2020-12-12,98379.9375
2020-12-19,19194.4375


### Gerando o dataset da bacia incremental

In [9]:
df = pd.merge(new_bacia_incremental, new_itaipu, left_index=True, right_index=True)
df

,bacia_prec_sum,vazao_itaipu
time,,
2001-01-06,94990.3125,96628.00
2001-01-13,56717.3750,113552.00
2001-01-20,50552.3750,84168.00
2001-01-27,55272.6250,81859.00
2001-02-03,84943.3125,98723.00
...,...,...
2020-12-05,44634.2500,37020.23
2020-12-12,98379.9375,46404.99
2020-12-19,19194.4375,63216.66


In [10]:
df.to_csv(f'{datasets}/Itaipu_POC_VAZAO_V3.csv')
pd.read_csv(f'{datasets}/Itaipu_POC_VAZAO_V3.csv', index_col='time')

,bacia_prec_sum,vazao_itaipu
time,,
2001-01-06,94990.3125,96628.00
2001-01-13,56717.3750,113552.00
2001-01-20,50552.3750,84168.00
2001-01-27,55272.6250,81859.00
2001-02-03,84943.3125,98723.00
...,...,...
2020-12-05,44634.2500,37020.23
2020-12-12,98379.9375,46404.99
2020-12-19,19194.4375,63216.66
